In [7]:
import boto3
import os
from imdb import IMDb

In [8]:
filepath = '/Users/tphoran/Movies/2018/'
directory_list = os.listdir(path=filepath)
year = '('+filepath[-5:-1]+')'

In [9]:
directory_list

['.DS_Store', 'Bohemian Rhapsody.m4v', 'The Grinch.m4v']

In [10]:
# Verify Movies Look Good
for i in directory_list:
    if (i[0] != '.'):
        ia = IMDb()
        m = i[:-4]+' '+year
        s_result = ia.search_movie(m)
        entry = s_result[0]
        ia.update(entry)
        print(entry['title'])
        print(entry['original air date'][:11])
        print(entry['plot'])
        print()

Bohemian Rhapsody
02 Nov 2018
['The story of the legendary rock band Queen and lead singer Freddie Mercury, leading up to their famous performance at Live Aid (1985).::Anonymous', 'Bohemian Rhapsody is a foot-stomping celebration of Queen, their music and their extraordinary lead singer Freddie Mercury. Freddie defied stereotypes and shattered convention to become one of the most beloved entertainers on the planet. The film traces the meteoric rise of the band through their iconic songs and revolutionary sound. They reach unparalleled success, but in an unexpected turn Freddie, surrounded by darker influences, shuns Queen in pursuit of his solo career. Having suffered greatly without the collaboration of Queen, Freddie manages to reunite with his bandmates just in time for Live Aid. While bravely facing a recent AIDS diagnosis, Freddie leads the band in one of the greatest performances in the history of rock music. Queen cements a legacy that continues to inspire outsiders, dreamers an

In [11]:
def build_s3_movies_list():
    s3 = boto3.resource('s3')
    s3_movie_list = []
    object_summary_iterator = s3.Bucket('manny-and-meche-plex').objects.all()
    for i in object_summary_iterator:
        if ('Movies/' in i.key) and (i.key!='Movies/'):
            s3_movie_list.append(i.key[7:])
    return s3_movie_list

In [12]:
# Add to S3 w/ Metadata
for i in directory_list:
    s3_movie_list = build_s3_movies_list()
    ia = IMDb()
    m = i[:-4]+' '+year
    s_result = ia.search_movie(m)
    entry = s_result[0]
    ia.update(entry)
    if (i not in s3_movie_list) and (i[0] != '.'):
        s3 = boto3.client('s3')
        s3.upload_file(filepath+i, \
                       'manny-and-meche-plex', \
                       'Movies/'+i,
                        ExtraArgs={'StorageClass':'ONEZONE_IA',
                                   'Metadata':{'movieID':entry.movieID,
                                               'proper_movie_title':i[:-4]}})
        print('Added File '+i)
    else:
        print('Already have '+i)

Already have .DS_Store
Already have Bohemian Rhapsody.m4v
Added File The Grinch.m4v
